In [1]:
import os 
import sys
base_dir = os.path.abspath('')
curr_dir = os.path.dirname(base_dir)
if not curr_dir in sys.path: sys.path.append(curr_dir)

In [2]:
import json

from chains.base import CommonChain
from embedding import initialize_embeddings
from models import initialize_llm
from router import PromptRouter
from utils import to_markdown

In [3]:
llm = initialize_llm("gpt4")
embeddings = initialize_embeddings()

In [4]:
chain_info = json.load(open("../chains/chain_info.json"))

In [5]:
finance_chain = CommonChain("../data/financing-a-stevens-education.csv", embeddings)
graudate_chain = CommonChain("../data/graduate-education.csv", embeddings)
undergraduate_chain = CommonChain("../data/undergraduate-education.csv", embeddings)
student_life = CommonChain("../data/student-life.csv", embeddings)
student_services = CommonChain("../data/student-services.csv", embeddings)
graudate_tution_chain = CommonChain(
    "../data/tuition-fees-and-other-expenses-for-graduate-students.csv", embeddings
)
undergraudate_tution_chain = CommonChain(
    "../data/tuition-fees-and-other-expenses-for-undergraduate-students.csv", embeddings
)

chains = {
    "financing_a_stevens_education": finance_chain,
    "tuition_fees_and_other_expenses_for_undergraduate_students": undergraudate_tution_chain,
    "tuition_fees_and_other_expenses_for_graduate_students": graudate_tution_chain,
    "student_life_at_stevens": student_life,
    "student_services_at_stevens": student_services,
    "graduate_education_at_stevens": graudate_chain,
    "undergraduate_chain": undergraduate_chain,
}

In [6]:
router = PromptRouter(llm=llm, chain_info=chain_info)

In [7]:
prompt = "What is the financial aid provided for graduate courses?"

In [8]:
chain_name = router.get_chain_name(prompt=prompt)
chain_name

'financing_a_stevens_education'

In [9]:
curr_chain = chains.get(chain_name)
response = curr_chain.process_prompt(llm, prompt=prompt)

In [10]:
response

{'context': [Document(page_content='\n\r\n\tGraduate Student Financial Information\r\n\n\nFinancial Aid Sources\n\n'),
  Document(page_content='\n\r\n\tFinancial Aid Sources\r\nMany sources of financial aid are available to graduate students. These include fellowships, assistantships, on-campus employment, employer tuition assistance plans, loan funds, and deferred payment plans. Fellowships and assistantships are granted on a competitive basis to outstanding full-time graduate students. Applicants should consult their department for more information regarding assistantships. Continuing students may become eligible for additional sources of aid as they progress through the program and should consult with their department at regular intervals.\nClick here for more information.\n\nGraduate AssistantshipsProvost Masters Fellowship AwardsResearch AssistantshipsTeaching AssistantshipsRobert Crooks Stanley Graduate FellowshipsGraduate GrantLoans and Work-StudyFAFSAFederal LoansPrivate Educat